In [1]:
!jupyter nbconvert --to slides main_workflow.ipynb

[NbConvertApp] Converting notebook main_workflow.ipynb to slides
[NbConvertApp] Writing 2980985 bytes to main_workflow.slides.html


# Use Case

In [2]:
import pandas as pd
import folium
import numpy as np
import seaborn as sns
import branca

df = pd.read_csv('data_cleaned.csv')
df['code_postal_tronque'] = df['code_postal'].apply(lambda x: x[:2] if len(x)==5 else x)

In [3]:
# Création de la carte
m = folium.Map(location=[47, 3], zoom_start=6)

# Ajout des cercles concentriques pour chaque code postal
for code_postal in df['code_postal'].unique():
    density = df.loc[df['code_postal'] == code_postal].shape[0]
    radius = np.log(float(density)**10) * 2
    color = df.loc[df['code_postal'] == code_postal, 'boutique'].iloc[0]
    
    folium.CircleMarker(
        location=[df.loc[df['code_postal'] == code_postal, 'lat'].mean(), df.loc[df['code_postal'] == code_postal, 'lon'].mean()],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)

# Affichage de la carte
m

In [4]:
len(df['boutique'].unique())

6

In [9]:
import folium
import numpy as np
import seaborn as sns

# Création de la carte
m = folium.Map(location=[47, 3], zoom_start=3)
m.get_root().title = "Carte des densités clients"

categorie = "boutique"

# Définition des couleurs pour chaque valeur unique de la colonne ciblée par f"{categorie}"
color_palette = sns.color_palette("husl", len(df[f'{categorie}'].unique())).as_hex()
cat_colors = dict(zip(df[f'{categorie}'].unique(), color_palette))

grouped_data = df.groupby(by=['boutique', 'code_postal', 'lat', 'lon', 'lat_boutique', 'lon_boutique', f'{category}']).size().reset_index(name='counts')


# Ajout des cercles concentriques pour chaque code postal
for code_postal in df['code_postal'].unique():
    density = df.loc[df['code_postal'] == code_postal].shape[0]
    radius = np.log(float(density)**10)
    color = cat_colors[df.loc[df['code_postal'] == code_postal, f'{categorie}'].iloc[0]]
    
    folium.CircleMarker(
        location=[df.loc[df['code_postal'] == code_postal, 'lat'].mean(), df.loc[df['code_postal'] == code_postal, 'lon'].mean()],
        radius=1,
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)
    
# Créer une ligne entre le client et la boutique
for i, row in grouped_data.iterrows():
    folium.PolyLine(
        locations=[[row['lat_boutique'], row['lon_boutique']], [row['lat'], row['lon']]],
        color=boutique_colors[df.loc[df['code_postal'] == row.code_postal, 'boutique'].iloc[0]],
        weight=density,
        opacity=0.2
    ).add_to(m)

m


In [11]:
#TEST 3

import folium
import numpy as np
import seaborn as sns
import matplotlib

# Création de la carte
m = folium.Map(location=[47, 3],zoom_start=6, zoom_control=False)

# Choix de la catégorie
category = "produit_prefere"

# Définition des couleurs pour chaque valeur unique de la colonne "boutique"
boutique_color_palette = ['blue','orange','green','purple','red','darkred']
boutique_colors = dict(zip(df['boutique'].unique(), boutique_color_palette))

color_palette = sns.color_palette("husl", len(df[f'{category}'].unique())).as_hex()
cat_colors = dict(zip(df[f'{category}'].unique(), color_palette))


grouped_data = df.groupby(by=['boutique', 'code_postal', 'lat', 'lon', 'lat_boutique', 'lon_boutique', f'{category}']).size().reset_index(name='counts')

#Ajout des cercles pour chaque groupe
lgd_txt = '<span style="color: {col};">{txt}</span>'
for i, row in grouped_data.iterrows():
    fg = folium.FeatureGroup(name=lgd_txt.format( txt= f'{row[category]}', col= cat_colors[row[f'{category}']]))
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=np.log(float(row['counts'])**10),
        color=cat_colors[row[f'{category}']],
        popup=row[f'{category}'],
        fill=True
    ).add_to(m)
    m.add_child(fg)

# Mettre un marqueur spécial pour chaque boutique
for boutique in df.boutique.unique():
    folium.Marker(
        location=[df[df.boutique == boutique]['lat_boutique'].mean(),
        df[df.boutique == boutique]['lon_boutique'].mean()],
        popup=boutique,
        icon=folium.Icon(color=boutique_colors[boutique],
        icon='fa-regular fa-cookie-bite')
    ).add_to(m)

folium.map.LayerControl('topright', collapsed= False).add_to(m)

m